# 회귀 모델 구축: 데이터 준비 및 시각화

## **호박을 위한 선형 회귀 - 2강**
#### 소개

이제 Tidymodels와 Tidyverse를 사용하여 머신러닝 모델을 구축할 준비가 되었으니, 데이터를 탐구하며 질문을 시작할 차례입니다. 데이터를 다루고 머신러닝 솔루션을 적용할 때, 데이터셋의 잠재력을 제대로 발휘하기 위해 올바른 질문을 하는 방법을 이해하는 것이 매우 중요합니다.

이 강의에서 배우게 될 내용:

- 모델 구축을 위해 데이터를 준비하는 방법.

- `ggplot2`를 사용하여 데이터를 시각화하는 방법.

답을 얻고자 하는 질문은 어떤 유형의 머신러닝 알고리즘을 사용할지 결정하게 됩니다. 그리고 얻는 답변의 품질은 데이터의 특성에 크게 좌우됩니다.

실제 연습을 통해 이를 살펴보겠습니다.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>@allison_horst의 작품</figcaption>


<!--![Artwork by \@allison_horst](../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.ko.jpg)<br>@allison_horst의 작품-->


## 1. 호박 데이터 가져오기와 Tidyverse 호출하기

이 강의를 진행하기 위해 다음 패키지가 필요합니다:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/)는 데이터 과학을 더 빠르고, 쉽고, 재미있게 만들어주는 [R 패키지 모음](https://www.tidyverse.org/packages)입니다!

다음 명령어로 설치할 수 있습니다:

`install.packages(c("tidyverse"))`

아래 스크립트는 이 모듈을 완료하는 데 필요한 패키지가 설치되어 있는지 확인하고, 누락된 경우 자동으로 설치해줍니다.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

이제 몇 가지 패키지를 실행하고 이 강의를 위해 제공된 [데이터](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv)를 로드해 봅시다!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

`glimpse()`를 사용하면 데이터에 빈칸이 있고 문자열(`chr`)과 숫자 데이터(`dbl`)가 섞여 있다는 것을 바로 확인할 수 있습니다. `Date`는 문자형 데이터이고, `Package`라는 이상한 열도 있는데, 이 열의 데이터는 `sacks`, `bins` 및 기타 값들이 섞여 있습니다. 사실, 데이터가 꽤 엉망입니다 😤.

사실, 완전히 준비된 상태의 데이터셋을 받아서 바로 ML 모델을 만들 수 있는 경우는 드뭅니다. 하지만 걱정하지 마세요. 이번 레슨에서는 표준 R 라이브러리를 사용하여 원시 데이터셋을 준비하는 방법을 배우게 됩니다 🧑‍🔧. 또한 데이터를 시각화하는 다양한 기술도 배우게 될 것입니다.📈📊
<br>

> 복습: 파이프 연산자 (`%>%`)는 객체를 함수나 호출 표현식으로 전달하여 논리적 순서로 작업을 수행합니다. 코드에서 파이프 연산자는 "그리고 나서"라고 말하는 것과 같은 역할을 한다고 생각할 수 있습니다.


## 2. 누락된 데이터 확인하기

데이터 과학자들이 자주 직면하는 문제 중 하나는 불완전하거나 누락된 데이터입니다. R은 누락되었거나 알 수 없는 값을 특별한 센티널 값인 `NA`(Not Available)로 나타냅니다.

그렇다면 데이터 프레임에 누락된 값이 포함되어 있는지 어떻게 알 수 있을까요?  
<br>
-   가장 간단한 방법은 기본 R 함수인 `anyNA`를 사용하는 것입니다. 이 함수는 논리 객체 `TRUE` 또는 `FALSE`를 반환합니다.


In [ ]:
pumpkins %>% 
  anyNA()

좋아요, 일부 데이터가 누락된 것 같네요! 그것부터 시작하면 좋을 것 같습니다.

-   또 다른 방법은 `is.na()` 함수를 사용하는 것으로, 이 함수는 각 열 요소 중 누락된 데이터를 논리값 `TRUE`로 표시합니다.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

이와 같은 큰 데이터 프레임에서는 모든 행과 열을 개별적으로 검토하는 것은 비효율적이고 사실상 불가능합니다😴.

-   더 직관적인 방법은 각 열의 결측값 합계를 계산하는 것입니다:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

훨씬 좋아졌습니다! 데이터가 일부 누락되었지만, 작업에는 큰 영향을 미치지 않을 수도 있습니다. 추가 분석 결과를 지켜보죠.

> R은 훌륭한 패키지와 함수 세트뿐만 아니라 매우 좋은 문서를 제공합니다. 예를 들어, `help(colSums)` 또는 `?colSums`를 사용하여 해당 함수에 대해 더 알아볼 수 있습니다.


## 3. Dplyr: 데이터 조작을 위한 문법

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>@allison_horst의 작품</figcaption>


<!--![@allison_horst의 작품](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.ko.png)<br/>@allison_horst의 작품-->


[`dplyr`](https://dplyr.tidyverse.org/)은 Tidyverse에 포함된 패키지로, 데이터 조작을 위한 문법을 제공하며 가장 일반적인 데이터 조작 문제를 해결할 수 있도록 일관된 동사를 제공합니다. 이 섹션에서는 dplyr의 동사들에 대해 알아보겠습니다!  
<br>


#### dplyr::select()

`select()`는 `dplyr` 패키지에 포함된 함수로, 유지하거나 제외할 열을 선택하는 데 도움을 줍니다.

데이터 프레임을 더 다루기 쉽게 만들기 위해, `select()`를 사용하여 필요한 열만 남기고 여러 열을 제거할 수 있습니다.

예를 들어, 이번 연습에서는 `Package`, `Low Price`, `High Price`, `Date` 열을 분석에 사용할 것입니다. 이 열들을 선택해 봅시다.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()`는 `dplyr` 패키지에 있는 함수로, 기존 열을 유지하면서 새로운 열을 생성하거나 기존 열을 수정할 수 있도록 도와줍니다.

`mutate`의 일반적인 구조는 다음과 같습니다:

`data %>%   mutate(new_column_name = what_it_contains)`

`Date` 열을 사용하여 다음 작업을 수행하며 `mutate`를 직접 사용해 보겠습니다:

1. 날짜(현재 문자형)를 월 형식으로 변환합니다(이 날짜는 미국 형식이므로 `MM/DD/YYYY` 형식입니다).

2. 날짜에서 월을 추출하여 새로운 열에 저장합니다.

R에서는 [lubridate](https://lubridate.tidyverse.org/) 패키지가 날짜-시간 데이터를 더 쉽게 다룰 수 있도록 도와줍니다. 따라서 `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()`를 사용하여 위의 목표를 달성하는 방법을 살펴보겠습니다. 이후 작업에서 더 이상 필요하지 않으므로 Date 열은 삭제할 수 있습니다.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

우와! 🤩

다음으로, 호박의 평균 가격을 나타내는 새로운 열 `Price`를 만들어 봅시다. 이제 `Low Price`와 `High Price` 열의 평균을 계산하여 새로운 Price 열을 채워보세요.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

예스!💪

"잠깐만요!", 당신은 `View(pumpkins)`로 전체 데이터 세트를 훑어본 후 말할 겁니다. "여기 뭔가 이상한 게 있어요!"🤔

`Package` 열을 보면, 호박은 여러 가지 다른 구성으로 판매됩니다. 일부는 `1 1/9 bushel` 단위로, 일부는 `1/2 bushel` 단위로, 일부는 호박 개수로, 일부는 무게 단위로, 그리고 일부는 폭이 다양한 큰 상자에 담겨 판매됩니다.

이것을 확인해 봅시다:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

놀라워요!👏

호박은 일관되게 무게를 재기가 매우 어려운 것 같으니, `Package` 열에 *bushel*이라는 문자열이 포함된 호박만 선택하여 새로운 데이터 프레임 `new_pumpkins`에 넣어 봅시다.
<br>


#### dplyr::filter()와 stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): 조건을 만족하는 **행**만 포함하는 데이터의 부분 집합을 생성합니다. 이 경우, `Package` 열에 *bushel* 문자열이 포함된 호박 데이터를 필터링합니다.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): 문자열에서 특정 패턴의 존재 여부를 감지합니다.

[`stringr`](https://github.com/tidyverse/stringr) 패키지는 일반적인 문자열 작업을 위한 간단한 함수들을 제공합니다.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

우리는 약 415줄 정도의 데이터를 좁혀서 한 바구니에 담긴 호박 데이터를 확인할 수 있습니다.🤩  
<br>


#### dplyr::case_when()

**하지만 잠깐! 해야 할 일이 하나 더 있습니다**

행마다 부셸 양이 다르다는 것을 눈치채셨나요? 가격을 1 1/9 부셸이나 1/2 부셸이 아닌, 부셸당 가격으로 표준화해야 합니다. 이제 수학을 활용해 이를 표준화할 시간입니다.

[`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) 함수를 사용하여 조건에 따라 Price 열을 *변경(mutate)*할 것입니다. `case_when`은 여러 `if_else()` 문을 벡터화하여 처리할 수 있도록 도와줍니다.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

이제 우리는 부셸 측정을 기반으로 단위당 가격을 분석할 수 있습니다. 하지만 이 모든 호박 부셸에 대한 연구는 `데이터의 본질을 이해하는 것`이 얼마나 `중요한지`를 보여줍니다!

> ✅ [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308)에 따르면, 부셸의 무게는 부피 측정 단위이기 때문에 농산물의 종류에 따라 달라집니다. "예를 들어, 토마토 한 부셸은 56파운드로 간주됩니다... 잎채소와 같은 경우는 공간을 더 많이 차지하지만 무게는 적기 때문에 시금치 한 부셸은 20파운드에 불과합니다." 꽤 복잡하죠! 부셸을 파운드로 변환하려고 애쓰지 말고, 부셸 단위로 가격을 매기는 게 낫습니다. 하지만 이 모든 호박 부셸에 대한 연구는 데이터의 본질을 이해하는 것이 얼마나 중요한지 다시 한번 보여줍니다!
>
> ✅ 반 부셸 단위로 판매되는 호박이 매우 비싸다는 것을 눈치채셨나요? 왜 그런지 알아낼 수 있나요? 힌트: 작은 호박이 큰 호박보다 훨씬 비쌉니다. 아마도 큰 속이 빈 파이 호박 하나가 차지하는 공간 때문에, 부셸당 훨씬 더 많은 작은 호박이 들어가기 때문일 것입니다.


이제 마지막으로, 모험심을 발휘해 💁‍♀️, Month 열을 첫 번째 위치로 옮겨봅시다. 즉, `Package` 열 `앞`으로 이동시키는 것입니다.

`dplyr::relocate()`는 열의 위치를 변경하는 데 사용됩니다.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

좋아요!👌 이제 새로운 회귀 모델을 구축할 수 있는 깔끔하고 정돈된 데이터셋이 준비되었습니다!  
<br>


## 4. ggplot2를 활용한 데이터 시각화

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>다사니 마디팔리의 인포그래픽</figcaption>


<!--![다사니 마디팔리의 인포그래픽](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.ko.png){width="600"}-->

다음과 같은 *현명한* 격언이 있습니다:

> "단순한 그래프는 데이터 분석가의 머릿속에 다른 어떤 도구보다 더 많은 정보를 전달해준다." --- 존 튜키

데이터 과학자의 역할 중 하나는 자신이 다루고 있는 데이터의 품질과 특성을 보여주는 것입니다. 이를 위해, 데이터 과학자들은 종종 데이터의 다양한 측면을 보여주는 흥미로운 시각화, 즉 플롯, 그래프, 차트를 만듭니다. 이렇게 하면, 다른 방법으로는 발견하기 어려운 관계나 격차를 시각적으로 보여줄 수 있습니다.

시각화는 또한 데이터에 가장 적합한 머신 러닝 기법을 결정하는 데 도움을 줄 수 있습니다. 예를 들어, 직선을 따르는 것처럼 보이는 산점도는 데이터가 선형 회귀 분석에 적합하다는 것을 나타냅니다.

R은 그래프를 만드는 여러 시스템을 제공하지만, [`ggplot2`](https://ggplot2.tidyverse.org/index.html)는 가장 우아하고 다재다능한 도구 중 하나입니다. `ggplot2`는 **독립적인 구성 요소를 결합**하여 그래프를 구성할 수 있게 해줍니다.

이제 Price와 Month 열을 사용하여 간단한 산점도를 만들어 봅시다.

이 경우, 먼저 [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html)를 사용하여 데이터셋과 미적 매핑을 제공하고 ([`aes()`](https://ggplot2.tidyverse.org/reference/aes.html) 사용), 그런 다음 산점도를 위한 레이어 ([`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html))를 추가합니다.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

이게 유용한 그래프일까요 🤷? 뭔가 놀라운 점이 있나요?

특별히 유용하지는 않습니다. 이 그래프는 특정 월에 데이터가 점으로 퍼져 있는 모습을 보여주는 것뿐입니다.
<br>


### **어떻게 유용하게 만들 수 있을까요?**

차트에 유용한 데이터를 표시하려면 데이터를 어떤 방식으로든 그룹화해야 하는 경우가 많습니다. 예를 들어, 우리의 경우 각 달의 호박 평균 가격을 찾으면 데이터의 기본 패턴에 대한 더 많은 통찰력을 제공할 수 있습니다. 이는 또 하나의 **dplyr** 기능을 사용하게 만듭니다:

#### `dplyr::group_by() %>% summarize()`

R에서 그룹화된 집계는 다음을 사용하여 쉽게 계산할 수 있습니다:

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()`는 분석 단위를 전체 데이터셋에서 개별 그룹(예: 월별)으로 변경합니다.

-   `dplyr::summarize()`는 그룹화 변수마다 하나의 열과 지정한 요약 통계마다 하나의 열을 포함하는 새로운 데이터 프레임을 생성합니다.

예를 들어, `dplyr::group_by() %>% summarize()`를 사용하여 **Month** 열을 기준으로 호박을 그룹화한 다음 각 달의 **평균 가격**을 찾을 수 있습니다.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

간단하게!✨

월과 같은 범주형 특징은 막대 그래프 📊로 표현하는 것이 더 적합합니다. 막대 그래프를 그리는 데 사용되는 레이어는 `geom_bar()`와 `geom_col()`입니다. 자세한 내용은 `?geom_bar`를 참고하세요.

하나 만들어볼까요!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩이것은 훨씬 더 유용한 데이터 시각화입니다! 이는 호박의 최고 가격이 9월과 10월에 발생한다는 것을 나타내는 것 같습니다. 이것이 당신의 예상과 일치하나요? 왜 그런가요, 아니면 왜 그렇지 않은가요?

두 번째 강의를 마친 것을 축하합니다 👏! 데이터를 모델 구축을 위해 준비한 후, 시각화를 통해 더 많은 인사이트를 발견했네요!



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서를 해당 언어로 작성된 상태에서 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.  
